In [18]:
import json
import requests
from bs4 import BeautifulSoup # parsing tool for wikipedia 
import sqlite3 as sql # database tool
import pandas as pd

page_url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films" # wikipedia page url
base_wiki_url = "https://en.wikipedia.org" # base url for accessing additional pages
response = requests.get(page_url) 
soup = BeautifulSoup(response.text, "html.parser")

In [19]:
tables = soup.find_all("table", class_="wikitable")
films_table = tables[0]

rows = films_table.find_all("tr")
films = []

# function to convert string representing revenue to int
def str2int(string: str) -> int:
    numeric = [char for char in string if char.isnumeric()]
    return int("".join(numeric))

# iterate through rows (except header row) and extract film data
for row in rows[1:]:
    cells = row.find_all("td")
    headers = row.find_all("th")

    film_href = headers[0].find_all("a")[0]

    # collect all available data from the row
    film = {
        "year": cells[3].text.strip(),
        "title": film_href.text.strip(),
        "revenue": str2int(cells[2].text.strip()),
        "href": film_href["href"], # we will use this to access director and country data
    }

    films.append(film)


In [ ]:
def save_to_csv(films, filename="highest_grossing_films.csv"):
    """Save the extracted film data to a CSV file."""
    df = pd.DataFrame(films)
    df.to_csv(filename, index=False)

# Main execution
save_to_csv(films)
print(f"Scraped data saved to 'highest_grossing_films.csv'")

Scraped data saved to 'highest_grossing_films.csv'
